In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('dataset_final.csv')

In [3]:
df

,Brand,Price,Sales,BSR,Active Sellers #,Ratings,Review Count,Images,Review velocity,Buy Box,...,Size Tier_Large Envelope,Size Tier_Large Letter,Size Tier_Large Oversize,Size Tier_Small Envelope,Size Tier_Small Oversize,Size Tier_Standard Envelope,Size Tier_Standard Parcel,Delivery_AMZ,Delivery_FBA,Delivery_MFN
0,789,0.592125,1277,59,-0.550521,5,-0.435086,1.010296,0.003233,38,...,0,0,1,0,0,0,0,1,0,0
1,278,-0.469718,10791,84,-0.550521,0,-0.612788,1.352821,0.003233,38,...,0,0,1,0,0,0,0,1,0,0
2,918,-0.343520,5111,179,1.281295,5,-0.101895,-0.359806,0.003233,38,...,0,0,0,0,1,0,0,1,0,0
3,894,-0.980881,99,15109,-0.550521,4,0.260913,-0.017280,0.003233,588,...,0,0,1,0,0,0,0,0,0,1
4,894,0.038896,122,7974,-0.550521,4,1.023550,0.325245,0.142447,588,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,364,-0.470992,233,6768,0.670690,5,0.327551,0.325245,0.003233,589,...,0,0,1,0,0,0,0,0,1,0
1998,257,-0.725936,227,8054,1.891900,5,0.727380,-0.359806,0.212053,38,...,0,0,1,0,0,0,0,1,0,0
1999,28,0.676256,340,6364,-0.550521,4,2.045335,1.010296,0.142447,22,...,0,0,1,0,0,0,0,0,1,0
2000,438,0.613795,162,6418,-0.550521,5,3.311460,-0.017280,0.142447,227,...,0,0,1,0,0,0,0,0,1,0


In [4]:
X = df.loc[:, df.columns!='Sales']
y = df[['Sales']]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=34)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1601, 26) (401, 26) (1601, 1) (401, 1)


## Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
 
model = RandomForestRegressor(random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [17]:
# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
 
mae, rmse, r2

(21.505436408977555, 53.17747579727319, 0.8743898163250485)

In [18]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

## MLP

In [102]:
import random
import tensorflow as tf

SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# MLP
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def build_model(num_input=1):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=num_input))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # activation = 'identity', 'logistic', 'tanh', 'relu'
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

model = build_model(num_input=26)

# mini bath
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2)

# evaluate
model.evaluate(X_test, y_test)

# cross validation
model = build_model(num_input=26)
history = model.fit(X_train, y_train, batch_size=32, epochs=200, validation_split=0.25, verbose=2)

Epoch 1/200
51/51 - 1s - loss: 814416.8125 - mae: 429.9390 - 1s/epoch - 27ms/step
Epoch 2/200
51/51 - 0s - loss: 6579810.0000 - mae: 725.1823 - 103ms/epoch - 2ms/step
Epoch 3/200
51/51 - 0s - loss: 181726.4375 - mae: 132.0487 - 103ms/epoch - 2ms/step
Epoch 4/200
51/51 - 0s - loss: 276000.7812 - mae: 213.0212 - 88ms/epoch - 2ms/step
Epoch 5/200
51/51 - 0s - loss: 125795.3672 - mae: 105.1978 - 110ms/epoch - 2ms/step
Epoch 6/200
51/51 - 0s - loss: 1193491.0000 - mae: 517.7007 - 111ms/epoch - 2ms/step
Epoch 7/200
51/51 - 0s - loss: 91278.7031 - mae: 75.3900 - 96ms/epoch - 2ms/step
Epoch 8/200
51/51 - 0s - loss: 66709.6250 - mae: 46.4907 - 106ms/epoch - 2ms/step
Epoch 9/200
51/51 - 0s - loss: 62372.9023 - mae: 40.2977 - 101ms/epoch - 2ms/step
Epoch 10/200
51/51 - 0s - loss: 59086.2070 - mae: 40.0681 - 88ms/epoch - 2ms/step
Epoch 11/200
51/51 - 0s - loss: 56751.0898 - mae: 42.6181 - 106ms/epoch - 2ms/step
Epoch 12/200
51/51 - 0s - loss: 59233.0312 - mae: 54.2367 - 113ms/epoch - 2ms/step
Epoc

In [103]:
y_pred = model.predict(X_test)
y_pred

13/13 [==============================] - 0s 3ms/step


array([[ 1.10064560e+02],
       [ 1.73234158e+01],
       [ 1.20335197e+01],
       [ 1.49278564e+01],
       [ 1.22598274e+02],
       [ 1.16291656e+01],
       [ 1.61993317e+02],
       [ 4.00593948e+00],
       [ 1.33348312e+01],
       [ 5.15557480e+00],
       [ 2.13261852e+01],
       [ 2.60551666e+02],
       [ 2.73427765e+02],
       [ 5.54477310e+00],
       [ 3.71873840e+02],
       [ 1.87420311e+01],
       [ 1.62942337e+02],
       [ 1.07323547e+03],
       [ 6.01062813e+01],
       [ 6.00191116e+00],
       [ 2.44948921e+01],
       [ 1.87231827e+00],
       [ 6.18508530e+00],
       [ 1.57796974e+01],
       [ 6.83614731e+00],
       [ 5.00187302e+00],
       [ 4.03654861e+00],
       [ 6.80505753e+00],
       [ 5.16615677e+00],
       [ 1.35985535e+02],
       [ 2.48128014e+01],
       [ 2.08653698e+01],
       [ 2.24563122e+01],
       [ 3.59374580e+01],
       [ 3.37666702e+00],
       [ 2.49339676e+00],
       [ 3.69852257e+01],
       [ 1.89922943e+01],
       [ 9.1

In [104]:
# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
 
mae, rmse, r2

(27.824841140214346, 67.21929688122366, 0.799295391701092)

## DNN

In [6]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from xgboost import XGBRegressor

In [25]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               3456      
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dense_7 (Dense)             (None, 256)               65792     
                                                                 
 dense_8 (Dense)             (None, 256)               65792     
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 168,321
Trainable params: 168,321
Non-trainable params: 0
_________________________________________________________________


In [26]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [27]:
NN_model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
35/40 [=========================>....] - ETA: 0s - loss: 195.3214 - mean_absolute_error: 195.3214
Epoch 1: val_loss improved from inf to 140.08327, saving model to Weights-001--140.08327.hdf5
40/40 [==============================] - 2s 15ms/step - loss: 193.9493 - mean_absolute_error: 193.9493 - val_loss: 140.0833 - val_mean_absolute_error: 140.0833
Epoch 2/500
33/40 [=======================>......] - ETA: 0s - loss: 89.0736 - mean_absolute_error: 89.0736 
Epoch 2: val_loss improved from 140.08327 to 85.76772, saving model to Weights-002--85.76772.hdf5
40/40 [==============================] - 0s 8ms/step - loss: 93.6011 - mean_absolute_error: 93.6011 - val_loss: 85.7677 - val_mean_absolute_error: 85.7677
Epoch 3/500
29/40 [====================>.........] - ETA: 0s - loss: 63.8005 - mean_absolute_error: 63.8005
Epoch 3: val_loss improved from 85.76772 to 72.26411, saving model to Weights-003--72.26411.hdf5
40/40 [==============================] - 0s 9ms/step - loss: 60.4992 

In [28]:
# Load wights file of the best model :
wights_file = 'Weights-012--50.37542.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [29]:
y_pred = NN_model.predict(X_test)
y_pred

13/13 [==============================] - 0s 5ms/step


array([[ 9.33632584e+01],
       [ 2.57763977e+01],
       [ 1.24536886e+01],
       [ 1.02707567e+01],
       [ 9.47497177e+01],
       [ 8.89441586e+00],
       [ 1.76847183e+02],
       [ 1.43474233e+00],
       [ 1.15040169e+01],
       [ 1.88322866e+00],
       [ 3.51107521e+01],
       [ 2.86823608e+02],
       [ 3.10230591e+02],
       [ 1.37755716e+00],
       [ 3.37032776e+02],
       [ 2.30733776e+01],
       [ 1.75000412e+02],
       [ 7.05648865e+02],
       [ 8.70865784e+01],
       [ 1.99848783e+00],
       [ 3.44419022e+01],
       [-9.31786120e-01],
       [ 3.36472750e+00],
       [ 1.31666880e+01],
       [ 4.15328789e+00],
       [ 1.87511432e+00],
       [ 9.15882528e-01],
       [ 4.13325787e+00],
       [ 1.71672094e+00],
       [ 1.23355888e+02],
       [ 4.07270241e+01],
       [ 2.66444187e+01],
       [ 3.52214470e+01],
       [ 5.14102936e+01],
       [-1.34368384e+00],
       [ 1.21117994e-01],
       [ 4.86644058e+01],
       [ 2.46498432e+01],
       [ 1.0

In [30]:
# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
 
mae, rmse, r2

(23.972583251524522, 56.37715002173767, 0.8588191966761305)

## KNN Regression

In [31]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor(n_neighbors=3)
knn_reg.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [34]:
y_pred = knn_reg.predict(X_test)
y_pred

array([[9.10000000e+01],
       [2.30000000e+01],
       [1.03333333e+01],
       [1.53333333e+01],
       [1.07333333e+02],
       [9.66666667e+00],
       [1.06000000e+02],
       [6.66666667e-01],
       [1.33333333e+01],
       [0.00000000e+00],
       [3.13333333e+01],
       [4.01666667e+02],
       [2.91666667e+02],
       [0.00000000e+00],
       [3.43000000e+02],
       [6.33333333e+00],
       [1.59666667e+02],
       [6.32333333e+02],
       [8.23333333e+01],
       [1.26666667e+01],
       [3.46666667e+01],
       [0.00000000e+00],
       [2.33333333e+00],
       [1.00000000e+01],
       [1.66666667e+00],
       [1.33333333e+00],
       [0.00000000e+00],
       [5.00000000e+00],
       [1.33333333e+00],
       [1.17333333e+02],
       [3.63333333e+01],
       [1.43333333e+01],
       [1.56666667e+01],
       [3.46666667e+01],
       [0.00000000e+00],
       [0.00000000e+00],
       [3.56666667e+01],
       [1.30000000e+01],
       [9.81666667e+02],
       [2.66666667e+00],


In [35]:
# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
 
mae, rmse, r2

(26.948462177888608, 66.33341269935978, 0.8045507104730336)